### ![](../../img/installation-ico.png) Installation 

In [ ]:
pip install langchain langchain-community duckduckgo-search

### ![](../../img/package-ico.png) Imports

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

import os
import google.auth
from langchain_google_vertexai import ChatVertexAI

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

from langchain import hub

### ![](../../img/parametrage-ico.png) Paramétrages des outils

In [ ]:
# Instantiation de l'outil de recherche

searchTool = DuckDuckGoSearchRun()

In [ ]:
# Informations sur l'outil

print(searchTool.name)
print(searchTool.description)
print(searchTool.args)

In [ ]:
# Création d'un outil custom

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

### ![](../../img/jouer-ico.png) Exécution des outils seuls

In [ ]:
# Utilisation de l'outil sans agent

searchTool.invoke("Who is Backack Obama ?")

In [ ]:
multiply.invoke({"a": 2, "b": 3})

### ![](../../img/parametrage-ico.png) Paramétrages de l'agent

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"

google.auth.default()

In [ ]:
# Création du prompt pour l'agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
    ]
)

In [ ]:
llm = ChatVertexAI(model_name="gemini-1.5-pro")
# Définition de la liste des outils

tools = [multiply, searchTool]

# Initialisation de l'agent à partir du LLM, des outils et du prompt
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Création du contexte d'exécution de l'agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# Exécution utilisant l'outil de recherche

agent_executor.invoke({"input": "Who is Backack Obama ?"})

In [ ]:
# Exécution utilisant l'outil de multiplication

agent_executor.invoke({"input": "What does 4 times 7 equal ?"})